In [12]:
import pandas as pd
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import tqdm
import time
import random
import re

In [3]:
# Load Spotify credentials
credentials_path = r'../creds/spotify_credentials.json'  # Path to your spotify_credentials.json file

with open(credentials_path, 'r') as file:
    creds = json.load(file)

# Initialize Spotify client
auth_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(auth_manager=auth_manager)

In [11]:
def get_playlist_info(sp, playlist_url):
    """Retrieve tracks, audio features, and metadata from a Spotify playlist URL."""
    # Extract playlist ID from URL
    playlist_id = playlist_url.split('/')[-1].split('?')[0]

    # Get playlist metadata
    playlist_metadata = sp.playlist(playlist_id)
    playlist_name = playlist_metadata['name']

    # Get playlist tracks
    playlist_tracks = sp.playlist_tracks(playlist_id)['items']
    tracks = []

    for track in playlist_tracks:
        track_name = track['track']['name']
        artists = []
        for artist in track['track']['artists']:
            artist_name = artist['name']
            artist_id = artist['id']
            artist_genres = sp.artist(artist_id)['genres']
            artists.append({'artist_name': artist_name, 'artist_genres': artist_genres})
        track_id = track['track']['id']

        # Get audio features for the track
        audio_features_dict = sp.audio_features(track_id)[0]
        audio_features_dict = {k: v for k, v in audio_features_dict.items() if k not in ['type', 'analysis_url', 'duration_ms']}

        tracks.append({
            'track_name': track_name,
            'artists': artists,
            'audio_features': audio_features_dict
        })

    return {
        'playlist_name': playlist_name,
        'tracks': tracks
    }

# Get playlist tracks and features
playlist_url = 'https://open.spotify.com/playlist/3B0Xfhq3aaUZk75wHAZrCs?si=eb8b4aaa1ec04ce3'
playlist = get_playlist_info(sp, playlist_url)
playlist

{'playlist_name': 'Superblue (123)',
 'tracks': [{'track_name': 'Ok Computer',
   'artists': [{'artist_name': 'Lemaitre', 'artist_genres': ['norwegian pop']},
    {'artist_name': 'RebMoe', 'artist_genres': []}],
   'audio_features': {'danceability': 0.947,
    'energy': 0.475,
    'key': 0,
    'loudness': -5.911,
    'mode': 1,
    'speechiness': 0.36,
    'acousticness': 0.0413,
    'instrumentalness': 4.77e-06,
    'liveness': 0.226,
    'valence': 0.711,
    'tempo': 117.974,
    'id': '0QgGXsIAWZYjfgw2g1Vjrs',
    'uri': 'spotify:track:0QgGXsIAWZYjfgw2g1Vjrs',
    'track_href': 'https://api.spotify.com/v1/tracks/0QgGXsIAWZYjfgw2g1Vjrs',
    'time_signature': 4}},
  {'track_name': 'Hot Wheels',
   'artists': [{'artist_name': 'Kaskade',
     'artist_genres': ['edm',
      'electro house',
      'pop dance',
      'progressive electro house',
      'progressive house',
      'vocal house']}],
   'audio_features': {'danceability': 0.814,
    'energy': 0.619,
    'key': 5,
    'loudnes

In [13]:
def select_first_song(playlist_data, randomize=True, track_name=None):
    """
    Selects the first song from the provided playlist data.

    Args:
        playlist_data (dict): A dictionary containing the playlist name and tracks.
        randomize (bool, optional): If True, randomly selects the first song. If False, requires a track_name.
        track_name (str, optional): The name of the track to select as the first song. Required if randomize is False.

    Returns:
        dict: A dictionary representing the first song in the setlist.

    Raises:
        ValueError: If randomize is False and track_name is not provided or not found in the playlist.
    """
    tracks = playlist_data['tracks']

    if randomize:
        first_song = random.choice(tracks)
    else:
        if not track_name:
            raise ValueError("If randomize is False, track_name must be provided.")

        for track in tracks:
            if track['track_name'] == track_name:
                first_song = track
                break
        else:
            raise ValueError(f"Track '{track_name}' not found in the playlist.")

    return first_song

first_song = select_first_song(playlist, randomize=True)
print(f"The first song in the setlist is: {first_song['track_name']}")

The first song in the setlist is: Going Down Slow


In [17]:
def get_camelot_key(key, mode):
    """
    Convert Spotify key and mode values to Camelot notation.

    Args:
        key (int): The key value from Spotify (0-11).
        mode (int): The mode value from Spotify (0 for minor, 1 for major).

    Returns:
        str: The key in Camelot notation (e.g., 'C', 'C#', 'Dm').
    """
    camelot_keys = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    camelot_key = camelot_keys[key]
    if mode == 0:
        camelot_key += 'm'
    return camelot_key

def get_next_song(playlist, current_song_idx, bpm_range=0.05, key_priority=['same', 'adjacent', 'fifth']):
    """Get the next song in the setlist based on BPM and key."""
    current_song = playlist['tracks'][current_song_idx]
    current_bpm = current_song['audio_features']['tempo']
    current_key = get_camelot_key(current_song['audio_features']['key'], current_song['audio_features']['mode'])
    
    # Filter songs within BPM range
    bpm_min = current_bpm * (1 - bpm_range)
    bpm_max = current_bpm * (1 + bpm_range)
    candidates = [song for i, song in enumerate(playlist['tracks']) if i != current_song_idx and bpm_min <= song['audio_features']['tempo'] <= bpm_max]
    
    # Sort candidates by key priority
    for priority in key_priority:
        if priority == 'same':
            key_candidates = [song for song in candidates if get_camelot_key(song['audio_features']['key'], song['audio_features']['mode']) == current_key]
            if key_candidates:
                return key_candidates
        elif priority == 'adjacent':
            adjacent_keys = [current_key[0] + str(int(current_key[1:]) + 1), current_key[0] + str(int(current_key[1:]) - 1)]
            key_candidates = [song for song in candidates if get_camelot_key(song['audio_features']['key'], song['audio_features']['mode']) in adjacent_keys]
            if key_candidates:
                return key_candidates
        elif priority == 'fifth':
            fifth_keys = [current_key[0] + str((int(current_key[1:]) + 7) % 12 + 1), current_key[0] + str((int(current_key[1:]) - 5) % 12 + 1)]
            key_candidates = [song for song in candidates if get_camelot_key(song['audio_features']['key'], song['audio_features']['mode']) in fifth_keys]
            if key_candidates:
                return key_candidates
    
    # If no match found, return the first candidate
    return candidates if candidates else None



In [18]:
def generate_setlist(playlist_data, randomize_first=True, track_name=None, bpm_range=0.05, key_priority=['same', 'adjacent', 'fifth']):
    """
    Generate a setlist from the provided playlist data.

    Args:
        playlist_data (dict): A dictionary containing the playlist name and tracks.
        randomize_first (bool, optional): If True, randomly selects the first song. If False, requires a track_name.
        track_name (str, optional): The name of the track to select as the first song. Required if randomize_first is False.
        bpm_range (float, optional): The range of BPM values to consider for the next song (e.g., 0.05 means ±5% of the current BPM).
        key_priority (list, optional): A list of priorities for selecting the next song based on key ('same', 'adjacent', 'fifth').

    Returns:
        list: A list of dictionaries representing the setlist, where each dictionary contains the track information.
    """
    setlist = []

    # Select the first song
    first_song = select_first_song(playlist_data, randomize=randomize_first, track_name=track_name)
    first_song_idx = playlist_data['tracks'].index(first_song)
    setlist.append(first_song)

    # Generate the rest of the setlist
    current_song_idx = first_song_idx
    while True:
        next_song = get_next_song(playlist_data, current_song_idx, bpm_range, key_priority)
        if next_song is None:
            break
        next_song_idx = playlist_data['tracks'].index(next_song)
        setlist.append(next_song)
        current_song_idx = next_song_idx

    return setlist

generate_setlist(playlist)



ValueError: invalid literal for int() with base 10: ''

In [20]:
first_song['track_name']

{'track_name': 'Going Down Slow',
 'artists': [{'artist_name': 'Exmag',
   'artist_genres': ['funk rock', 'livetronica']}],
 'audio_features': {'danceability': 0.61,
  'energy': 0.661,
  'key': 2,
  'loudness': -4.892,
  'mode': 1,
  'speechiness': 0.0423,
  'acousticness': 0.00281,
  'instrumentalness': 0.0222,
  'liveness': 0.123,
  'valence': 0.276,
  'tempo': 119.99,
  'id': '1IUZh86P1U3w8i245mhzJi',
  'uri': 'spotify:track:1IUZh86P1U3w8i245mhzJi',
  'track_href': 'https://api.spotify.com/v1/tracks/1IUZh86P1U3w8i245mhzJi',
  'time_signature': 4}}